In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import openai
import time

openai.api_key = "REPLACE WITH OPEN AI API KEY" #API key
chosen_model = "gpt-4o" #Choose your GPT model here

test_api_key = False #True
if test_api_key:
    try:
        openai.models.list()  # Test API call
        print("API key is valid.")
    except openai.error.AuthenticationError:
        print("Invalid API key.")
    except Exception as e:
        print(f"Error: {e}")
        
client = openai.OpenAI(api_key=openai.api_key)

c:\Users\ekabu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
df = pd.read_csv("subreddit_dataset_full_unlabeled.csv", on_bad_lines="skip")
print(df.head())
print(f"Shape of df: {df.shape}")

                                                Text
0  Forensic Audit: "We conclude that the Dominion...
1  Biden and The Squad's Tlaib stage heated confr...
2  President-elect Biden is projected to win Penn...
3  Not even an issue of politics it's wrong this ...
4  Young man wearing MAGA hat attacked at school ...
Shape of df: (14048, 1)


In [7]:
def batch_label_texts(texts):
    """Classifies multiple texts in a single API call."""
    prompt = f"""Classify the text into one of the following categories: Democratic, Republican, or NA.
    -Democratic: Supports Democratic party, its members/policies, or show negative sentiment of the opposition party/member/policies.
    -Republican: Supports Republican party, its members/policies, or show negative sentiment of the opposition party/member/policies.
    -NA: Neutral or unrelated.

    Provide labels in this format: 
    1. <category>
    2. <category>
    3. <category>
    ...
    
    Texts to classify:
    """ + "\n".join([f"{i+1}. {text}" for i, text in enumerate(texts)])

    for attempt in range(3):  # Retry logic
        try:
            response = client.chat.completions.create(
                model=chosen_model,
                messages=[{"role": "system", "content": "You are a helpful text classifier."},
                          {"role": "user", "content": prompt}],
                temperature=0
            )
            
            labels = response.choices[0].message.content.strip().split("\n")
            return [label.split(". ")[-1] for label in labels]  # Extract labels
        except openai.RateLimitError:
            print("Rate limit reached. Waiting before retrying...")
            time.sleep(10)
        except openai.APITimeoutError:
            print("Request timed out. Retrying...")
            time.sleep(5)
        except Exception as e:
            print(f"Error: {e}")
            return ["Error"] * len(texts)

    return ["Error"] * len(texts)  # If it fails after retries

In [8]:
# Process in batches
batch_size = 10
labeled_data = []

for i in range(0, len(df), batch_size):
    batch = df["Text"].iloc[i:i+batch_size].tolist()
    labels = batch_label_texts(batch)
    
    for text, label in zip(batch, labels):
        labeled_data.append({"Text": text, "Label": label})
    
    time.sleep(1)  # Reduce rate limit issues

In [9]:
# Save results
df_labeled = pd.DataFrame(labeled_data)
df_labeled.to_csv("subreddit_dataset_full_labeled_gpt4o.csv", index=False)

print("Labeling completed. Saved to .csv.")

Labeling completed. Saved to .csv.
